In [1]:
!pip install selenium


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  

driver.get("https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile")


Uthrapradesh_routes_links = []

def scrape_page():
    
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route")))

   
    for route in routescontainer:
        try:
            routename= route.text
            routelink= route.get_attribute('href')

          
            Uthrapradesh_routes_links.append({'routename':routename,'routelink':routelink})

        except Exception as e:
            print(f"An error occurred: {e}")
            continue
        


for page_number in range(1,6):
    scrape_page()
    if page_number < 5:  
        try:
           
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

           
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1) 

            
            print(f"Clicking on page {page_number + 1}")

            
            next_page_button.click()

            
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            
            print(f"Successfully navigated to page {page_number + 1}")

            
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Print the scraped data
for entry in Uthrapradesh_routes_links:
    print(entry)

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
{'routename': 'Delhi to Bareilly (Uttar Pradesh)', 'routelink': 'https://www.redbus.in/bus-tickets/delhi-to-bareilly'}
{'routename': 'Delhi to Lucknow', 'routelink': 'https://www.redbus.in/bus-tickets/delhi-to-lucknow'}
{'routename': 'Bareilly (Uttar Pradesh) to Delhi', 'routelink': 'https://www.redbus.in/bus-tickets/bareilly-to-delhi'}
{'routename': 'Lucknow to Gorakhpur (uttar pradesh)', 'routelink': 'https://www.redbus.in/bus-tickets/lucknow-to-gorakhpur'}
{'routename': 'Aligarh (uttar pradesh) to Delhi', 'routelink': 'https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-delhi'}
{'routename': 'Lucknow to Prayagraj(Uttar Pradesh)', 'routelink': 'https://www.redbus.in/bus-tickets/lucknow-to-allahabad'}
{'routename': 'Delhi to Aligarh (uttar pradesh)', 'routelink': 'https://w

In [2]:
import pandas as pd

In [3]:
import pandas as pd
pd.DataFrame(Uthrapradesh_routes_links).to_csv("Uthrapradesh_routes_links.csv",index=False)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

def setup_driver():
    """Set up and configure Chrome WebDriver"""
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-gpu')
    
    driver = webdriver.Chrome(options=options)
    return driver

def button_interaction_with_wait(driver, wait):
    """
    Click button and wait for 30 seconds before proceeding
    
    Args:
        driver (WebDriver): Selenium WebDriver instance
        wait (WebDriverWait): WebDriverWait instance for element waiting
    
    Returns:
        bool: True if interaction successful, False otherwise
    """
    try:
      
        button_element = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//*[@class='button']"))
        )
        
       
        button_element.click()
        
        
        print("Button clicked. Waiting for 30 seconds...")
        time.sleep(30)
        
        print("Wait complete. Proceeding with data scraping.")
        return True
    
    except Exception as e:
        print(f"Failed to interact with button: {e}")
        return False

def scrape_redbus_routes(url):
    """Scrape bus routes from RedBus"""
    driver = setup_driver()
    bus_routes = []

    try:
        
        driver.get(url)
        driver.maximize_window()
        
       
        wait = WebDriverWait(driver, 30)
        
        
        interaction_result = button_interaction_with_wait(driver, wait)
        
        if not interaction_result:
            print("Button interaction failed. Attempting to continue.")
        
        
        scroll_to_bottom(driver)
        
        
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
        
        
        routes_container = driver.find_elements(By.CLASS_NAME, "bus-item")
        
        
        for route in routes_container:
            try:
                bus_details = extract_bus_details(route)
                bus_routes.append(bus_details)
            except Exception as e:
                print(f"Error extracting route details: {e}")
        
        return bus_routes
    
    except Exception as e:
        print(f"An error occurred during scraping: {e}")
        return bus_routes
    
    finally:
       
        driver.quit()

def scroll_to_bottom(driver):
    """Scroll to the bottom of the page"""
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        time.sleep(2)
        
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def extract_bus_details(route):
    """Extract details for a single bus route"""
    def safe_find_text(element, by, value, default="Not Available"):
        """Safely find and return text of an element"""
        try:
            return element.find_element(by, value).text.strip()
        except NoSuchElementException:
            return default
   
    return {
        'bus_name': safe_find_text(route, By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color"),
        'departure_time': safe_find_text(route, By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold"),
        'arrival_time': safe_find_text(route, By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline"),
        'bus_type': safe_find_text(route, By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus"),
        'duration': safe_find_text(route, By.CSS_SELECTOR, ".dur.l-color.lh-24"),
        'star_rating': safe_find_text(route, By.CSS_SELECTOR, ".lh-18.rating.rat-green span"),
        'price': safe_find_text(route, By.CSS_SELECTOR, ".fare.d-block .f-19.f-bold"),
        'seat_availability': safe_find_text(route, By.CSS_SELECTOR, ".seat-left.m-top-30")
    }

def main():
   
    url = "https://www.redbus.in/bus-tickets/shahjahanpur-to-delhi?fromCityId=84362&toCityId=733&fromCityName=Shahjahanpur%20(Uttar%20Pradesh)&toCityName=Delhi&busType=Any&srcCountry=IND&destCountry=null&onward=02-Dec-2024"
    
    
    bus_routes = scrape_redbus_routes(url)
    
   
    for route in bus_routes:
        print(route)
    
    
    df = pd.DataFrame(bus_routes)    
    df.to_csv("shahjahanpur_delhi.csv", index=False)  
    print("Data saved to shahjahanpur_delhi.csv")    

if __name__ == "__main__":
    main()


Button clicked. Waiting for 30 seconds...
Wait complete. Proceeding with data scraping.
{'bus_name': 'UPSRTC - BRT0066', 'departure_time': '00:05', 'arrival_time': '08:35', 'bus_type': 'Ordinary Non AC Seater 2+3', 'duration': '08h 30m', 'star_rating': 'Not Available', 'price': '613', 'seat_availability': 'Not Available'}
{'bus_name': 'UPSRTC - SJP0608', 'departure_time': '06:30', 'arrival_time': '14:45', 'bus_type': 'Ordinary Non AC Seater 2+3', 'duration': '08h 15m', 'star_rating': 'Not Available', 'price': '634', 'seat_availability': 'Not Available'}
{'bus_name': 'UPSRTC - SJP0200', 'departure_time': '07:00', 'arrival_time': '14:20', 'bus_type': 'Ordinary Non AC Seater 2+3', 'duration': '07h 20m', 'star_rating': 'Not Available', 'price': '634', 'seat_availability': 'Not Available'}
{'bus_name': 'UPSRTC - SJP0203', 'departure_time': '08:02', 'arrival_time': '15:22', 'bus_type': 'Ordinary Non AC Seater 2+3', 'duration': '07h 20m', 'star_rating': 'Not Available', 'price': '584', 'seat_

In [ ]:
import  mysql.connector



mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
)
print(mydb)
mycursor = mydb.cursor(buffered=True)

In [ ]:
mycursor.execute("""
CREATE TABLE bus_routes (
    id INT AUTO_INCREMENT PRIMARY KEY,
    route_name TEXT,
    route_link TEXT,
    busname TEXT,
    bustype TEXT,
    departing_time TIME,
    duration TEXT,
    reaching_time TIME,
    star_rating FLOAT,
    price DECIMAL,
    seats_available INT
)
""")

In [ ]:
import pandas as pd
import mysql.connector

# Load data from CSV file
file_path = r'D:/Data science/selenium/selenium_project.ipynb/RED_BUS.csv'

# Read CSV without headers and assign custom column names
data = pd.read_csv(file_path, header=None, names=[
    'route_name', 'route_link', 'busname', 'bustype', 
    'departing_time', 'duration', 'reaching_time', 
    'star_rating', 'price', 'seats_available', 'extra_column'
])

# Establish MySQL connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="redbus"
)

# Create a cursor object
mycursor = mydb.cursor()

# SQL query to insert data (excluding id as it's AUTO_INCREMENT)
insert_query = """ 
INSERT INTO bus_routes (
    route_name, route_link, busname, bustype,
    departing_time, duration, reaching_time,
    star_rating, price, seats_available
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 
"""

try:
    # Select only the columns you need
    columns_to_use = [
        'route_name', 'route_link', 'busname', 'bustype', 
        'departing_time', 'duration', 'reaching_time', 
        'star_rating', 'price', 'seats_available'
    ]
    
    # Iterate through the DataFrame and insert each row
    for index, row in data.iterrows():
        # Prepare the row data
        row_data = [
            row['route_name'],
            row['route_link'],
            row['busname'],
            row['bustype'],
            row['departing_time'],
            row['duration'],
            row['reaching_time'],
            # Handle 'Not Available' for star rating
            None if row['star_rating'] == 'Not Available' else row['star_rating'],
            # Remove any non-numeric characters from price
            ''.join(filter(str.isdigit, str(row['price']))),
            # Extract only the numeric part of seats available
            ''.join(filter(str.isdigit, str(row['seats_available'])))
        ]
        
        # Execute the insert query
        mycursor.execute(insert_query, row_data)
    
    # Commit the transaction
    mydb.commit()
    print(f"Data has been inserted successfully. {len(data)} rows added.")

except mysql.connector.Error as err:
    print(f"MySQL Error: {err}")
    mydb.rollback()

except Exception as e:
    print(f"Unexpected error: {e}")
    mydb.rollback()

finally:
    # Close the cursor and connection
    mycursor.close()
    mydb.close()